# TC Profiling


In [1]:
# global settings
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import matplotlib.pyplot as plt
import numpy as np
import seaborn

## SFC / VNF Model

The first approach for a simplistic SFC / VNF performance model that can be used to play with different profiling approaches. As a first shot, the model tries to mimic the results from our NFV-SDN'17 paper [1]. The model focusses only on throughput as main metric for now. 


<small>
* [1] M. Peuster and H. Karl: Profile Your Chains, Not Functions: Automated Network Service Profiling in DevOps Environments. IEEE Conference on Network Function Virtualization and Software Defined Networks (NFV-SDN), Berlin, Germany. (2017) 
</small>

In [42]:
class NS(object):
    """
    A network service based on a linear SFC: f1 -> f2 -> ... -> fN 
    """
    
    def __init__(self, name, vnfs, alpha_func=lambda: 1.0):
        """
        name: name of service (string)
        vnfs: functions representing the VNF's CPU-time -> throughput mapping (list)
        alpha_func: function to generate the alpha values to scale the VNF performance (e.g. random gen.)
        """
        self.name = name
        self.vnfs = vnfs
        self.alphas = [alpha_func() for _ in self.vnfs]
        print("Initialized '{}' with alphas={}".format(self.name, self.alphas))   
        
    def _calc_vnf_tp(self, cpu_times):
        """
        calculate TP for each function in self.vnfs
        cpu_times: CPU time available for each VNF
        """
        assert len(cpu_times) == len(self.vnfs) == len(self.alphas)
        # calculate result for each vnf and multiply by corresponding alpha
        return [f(r) * a for f, r, a in zip(self.vnfs, cpu_times, self.alphas)]
           
    def get_total_tp(self, cpu_times):
        """
        calculate TP of SFC
        cpu_times: CPU time available for each VNF
        """
        # uses "naive" minimum-TP model from NFV-SDN'17 paper for now
        return min(self._calc_vnf_tp(cpu_times))


In [43]:
#
# Model tests
#
# test with simple linear models (based on NFV-SDN'17 paper results)
vnfs = [
    lambda x: 8.0 * x,
    lambda x: 3.0 * x,
    lambda x: 1.2 * x
]

def uniform_rnd():
    return np.random.uniform(0.2, 5.0)

n1 = NS("n1", vnfs)
print(n1.get_total_tp([0.16, 0.16, 0.16]))
print(n1.get_total_tp([1.0, 1.0, 1.0]))
print(n1.get_total_tp([1.0, 1.0, 1.0]))

n2 = NS("n2", vnfs, alpha_func=uniform_rnd)
print(n2.get_total_tp([0.16, 0.16, 0.16]))
print(n2.get_total_tp([1.0, 1.0, 1.0]))
print(n2.get_total_tp([1.0, 1.0, 1.0]))

Initialized 'n1' with alphas=[1.0, 1.0, 1.0]
0.192
1.2
1.2
Initialized 'n2' with alphas=[2.0662965758804157, 3.9969666362217366, 4.904328269091169]
0.9416310276655045
5.885193922909403
5.885193922909403
